In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
repository = 'evaluating_factuality_word_definitions'

%cd /content/drive/My Drive/{repository}

Mounted at /content/drive
/content/drive/My Drive/evaluating_factuality_word_definitions


In [2]:
!pip install datasets
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 16.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-

In [3]:
import torch
import gc
import numpy as np
from datasets import Dataset
from peft import LoraConfig, TaskType, get_peft_model
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
from torch import optim
from tqdm import tqdm
import transformers
from transformers import AutoTokenizer, BigBirdModel
from matplotlib import pyplot as plt
from torch.cuda.amp import GradScaler, autocast

from config import DB_URL
from dataset.def_dataset import DefinitionDataset
from models.evidence_selection_model import EvidenceSelectionModel
from losses.supcon import SupConLoss
import torch.nn.functional as F

# Evaluation

In [4]:
def convert_to_labels(similarities, labels, k=2):
    top_indices = torch.topk(similarities, k=min(k, similarities.size(1)))[1]
    predicted = torch.zeros_like(similarities)
    predicted.scatter_(1, top_indices, 1)

    top_k_hits = labels[torch.arange(labels.size(0)).unsqueeze(1), top_indices]
    top_k_hits = torch.any(top_k_hits == 1, dim=1).float()

    mask = (labels != -1).flatten()
    return predicted.flatten()[mask], labels.flatten()[mask], top_k_hits

In [5]:
def evaluate(ev_model, dataloader, loss_function):
    gt_labels = []
    pr_labels = []
    all_top_k_hits = []
    all_loss = []
    for batch in tqdm(dataloader):
        ev_model.eval()
        with torch.no_grad():
            model_input = batch["model_input"]
            claim_embedding = ev_model(input_ids=model_input['claim_input_ids'],
                                       attention_mask=model_input['claim_attention_mask'])
            sentence_embeddings = ev_model(input_ids=model_input['input_ids'],
                                           attention_mask=model_input['attention_mask'],
                                           sentence_mask=model_input['sentence_mask'])

            loss = loss_function(claim_embedding, sentence_embeddings, labels=batch['labels'])
            claim_similarities = F.cosine_similarity(claim_embedding, sentence_embeddings, dim=2)
            claim_similarities = claim_similarities.nan_to_num(nan=float('-inf'))
        predicted, true_labels, top_k_hits = convert_to_labels(claim_similarities, batch['labels'],
                                                               k=3)
        gt_labels.extend(true_labels.tolist())
        pr_labels.extend(predicted.tolist())
        all_top_k_hits.extend(top_k_hits.tolist())
        all_loss.append(loss)
    loss = sum(all_loss) / len(all_loss)
    top_k_acc = sum(all_top_k_hits) / len(all_top_k_hits)

    return loss.item(), top_k_acc, classification_report(gt_labels, pr_labels)

# Training

In [12]:
torch.cuda.empty_cache()
gc.collect()

99

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = 'google/bigbird-roberta-large'
model = BigBirdModel.from_pretrained(model_name)

# Add all lora compatible modules
target_modules = []
for name, module in model.named_modules():
    if isinstance(module, (torch.nn.Linear, torch.nn.Embedding, torch.nn.Conv2d, transformers.pytorch_utils.Conv1D)):
        target_modules.append(name)

peft_config = LoraConfig(task_type=TaskType.FEATURE_EXTRACTION, inference_mode=False, r=64, lora_alpha=32, lora_dropout=0.1, target_modules=target_modules)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

selection_model = EvidenceSelectionModel(model).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)

trainable params: 32,124,416 || all params: 391,248,384 || trainable%: 8.21074726790437


In [10]:
dataset_query = """
select dd.id, dd.claim, dd.label, docs.document_id, docs.text,
       docs.lines, group_concat(dd.evidence_sentence_id) as evidence_lines
from def_dataset dd
    join documents docs on docs.document_id = dd.evidence_wiki_url
where set_type='{set_type}' -- and length(claim) < 50 and length(docs.text) < 400
group by dd.id, evidence_annotation_id, evidence_wiki_url
limit 2000
"""

train_dataset_raw = Dataset.from_sql(dataset_query.format(set_type='train'), con=DB_URL)
dev_dataset_raw = Dataset.from_sql(dataset_query.format(set_type='dev'), con=DB_URL)

train_dataset = DefinitionDataset(train_dataset_raw, tokenizer, mode='train', model='evidence_selection')
train_dataloader = DataLoader(train_dataset, shuffle=True,
                              collate_fn=train_dataset.collate_fn,
                              batch_size=1)
dev_dataset = DefinitionDataset(dev_dataset_raw, tokenizer, mode='train', model='evidence_selection')
dev_dataloader = DataLoader(train_dataset, shuffle=True,
                              collate_fn=dev_dataset.collate_fn,
                              batch_size=1)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1513 [00:00<?, ? examples/s]

In [13]:
#warmup_steps = 0
#t_total = int(len(train_dataloader) * args.num_epochs / args.gradient_accumulation_steps)

optimizer = optim.AdamW(selection_model.parameters(), lr=1e-5)
#scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
criterion = SupConLoss(temperature=0.17)

## Gradient Accumulation

Accumulates the gradient, which allows finetuning in a batch manner of large models. Works well for Bert

In [ ]:
timestamp = datetime.now().strftime("%m-%d_%H-%M")

num_epochs = 5
patience = 4
gradient_accumulation = 16  # 2048
trace_train = []
trace_val = []

selection_model.zero_grad()
use_amp = True
scaler = GradScaler(enabled=use_amp, init_scale=1)

best_loss = np.inf
for epoch in range(num_epochs):
    bar_desc = "Epoch %d of %d | Iteration" % (epoch + 1, num_epochs)
    train_iterator = tqdm(train_dataloader, desc=bar_desc)

    train_loss = 0
    print('Train ...')
    for step, batch in enumerate(train_iterator):
        selection_model.train()
        model_input = batch["model_input"]

        with autocast():
            claim_embedding = selection_model(input_ids=model_input['claim_input_ids'],
                                              attention_mask=model_input['claim_attention_mask'])
            sentence_embeddings = selection_model(input_ids=model_input['input_ids'],
                                                  attention_mask=model_input['attention_mask'],
                                                  sentence_mask=model_input['sentence_mask'])

            loss = criterion(claim_embedding, sentence_embeddings, labels=batch['labels'])
            train_loss += loss.detach().item()
            loss = (loss / gradient_accumulation)

        scaler.scale(loss).backward()

        total_norm = 0
        for name, param in selection_model.named_parameters():
            if param.grad is not None:
                param_norm = param.grad.data.norm(2)
                total_norm += param_norm.item() ** 2
        total_norm = total_norm ** (1 / 2)

        # Print gradient norm
        print(f'Gradient norm: {total_norm}')

        if (step + 1) % gradient_accumulation == 0:
            scaler.unscale_(optimizer)
            count = 0
            scaler.step(optimizer)
            scaler.update()
            optimizer.step()
            # scheduler.step()
            optimizer.zero_grad()

    trace_train.append(train_loss / len(train_dataloader))
    # validation
    with torch.no_grad():
        val_loss, val_top_k_acc, report = evaluate(selection_model, dev_dataloader, criterion)
        trace_val.append(val_loss)
        print(
            f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {(train_loss / len(train_dataloader)):.4f}, Validation Loss: {val_loss:.4f}')
        print(f'Validation top k acc: {val_top_k_acc:.4f}')
        print(report)

        if val_loss < best_loss:
            best_loss = val_loss
            best_epoch = epoch
            best_state = {key: value.cpu() for key, value in
                          selection_model.state_dict().items()}
            selection_model.save(f'selection_model_intermediate_{timestamp}')
        else:
            if epoch >= best_epoch + patience:
                break

selection_model.load_state_dict(best_state)
selection_model.save(f'selection_model_{timestamp}')

plt.plot(trace_train, label='train')
plt.plot(trace_val, label='validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

Epoch 1 of 5 | Iteration:   0%|          | 0/1212 [00:00<?, ?it/s]Attention type 'block_sparse' is not possible if sequence_length: 23 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Train ...


Epoch 1 of 5 | Iteration:   0%|          | 1/1212 [00:01<27:30,  1.36s/it]

Gradient norm: 23.391392473183185


Epoch 1 of 5 | Iteration:   0%|          | 2/1212 [00:01<18:00,  1.12it/s]

...

Epoch 5 of 5 | Iteration:   9%|▉         | 111/1212 [00:55<09:15,  1.98it/s]

Gradient norm: 315.6705441520727


Epoch 5 of 5 | Iteration:   9%|▉         | 112/1212 [00:56<09:37,  1.90it/s]

Gradient norm: 315.95795195055535


Epoch 5 of 5 | Iteration:   9%|▉         | 113/1212 [00:56<09:57,  1.84it/s]

In [ ]:
test_dataset_raw = Dataset.from_sql(dataset_query.format(set_type='test'), con=DB_URL)
test_dataset = DefinitionDataset(test_dataset_raw, tokenizer, mode='train', model='evidence_selection')
test_dataloader = DataLoader(test_dataset, shuffle=True, collate_fn=test_dataset.collate_fn, batch_size=10)

test_loss = evaluate(model, test_dataloader)
print(test_loss)

In [10]:
print(trace_train)
print(len(train_dataloader))

[]
581
